# Starboard News — Partner Integration Guide

This notebook demonstrates how a partner would integrate the Starboard News API into their website or app.

**Base URL:** `https://news.starboard.to`

**Authentication:** Public endpoints require no auth. Private endpoints require `Bearer CRON_SECRET`.

In [1]:
import requests
import json
from datetime import date

BASE_URL = "https://news.starboard.to"

def pp(resp):
    """Pretty print API response"""
    print(f"HTTP {resp.status_code}")
    try:
        print(json.dumps(resp.json(), indent=2, ensure_ascii=False))
    except:
        print(resp.text[:500])

---
## Step 1: Discover Available Newsletters

First, fetch the list of newsletters to display on your website.
This is public — no auth needed.

In [2]:
resp = requests.get(f"{BASE_URL}/api/newsletters")
pp(resp)

# Parse newsletter options for use in a UI
newsletters = resp.json()["newsletters"]
print(f"\nAvailable newsletters: {len(newsletters)}")
for n in newsletters:
    print(f"  {n['emoji']} {n['id']:20s} {n['name']} ({n['language']})")

HTTP 200
{
  "newsletters": [
    {
      "id": "minor_news",
      "name": "Minor News",
      "description": "Daily crypto & energy infrastructure news digest",
      "emoji": "⚡",
      "language": "en"
    },
    {
      "id": "into_crypto_cn",
      "name": "Into Crypto 中文版",
      "description": "每日加密货币深度分析",
      "emoji": "🪙",
      "language": "zh"
    },
    {
      "id": "into_crypto_en",
      "name": "Into Crypto",
      "description": "Daily crypto analysis and insights",
      "emoji": "🪙",
      "language": "en"
    }
  ]
}

Available newsletters: 3
  ⚡ minor_news           Minor News (en)
  🪙 into_crypto_cn       Into Crypto 中文版 (zh)
  🪙 into_crypto_en       Into Crypto (en)


---
## Step 2: Subscribe a User

When a user fills in their email and selects newsletters on your site,
make this POST request. The user will immediately receive the latest issue.

In [3]:
# Subscribe a user to two newsletters
resp = requests.post(f"{BASE_URL}/api/subscribe", json={
    "email": "partner-test@example.com",
    "feeds": ["minor_news", "into_crypto_en"]
})
pp(resp)

HTTP 200
{
  "success": true,
  "message": "Subscribed! Check your inbox.",
  "feeds": [
    "minor_news",
    "into_crypto_en"
  ]
}


In [ ]:
# Re-subscribing the same user with a new feed — merges, doesn't duplicate
resp = requests.post(f"{BASE_URL}/api/subscribe", json={
    "email": "partner-test@example.com",
    "feeds": ["into_crypto_cn"]
})
pp(resp)
# Notice: feeds now includes all 3

### Error Handling

Here's what your frontend should handle:

In [ ]:
# Invalid email
resp = requests.post(f"{BASE_URL}/api/subscribe", json={
    "email": "not-an-email",
    "feeds": ["minor_news"]
})
print("Invalid email ->")
pp(resp)
assert resp.status_code == 400

In [ ]:
# Invalid feed ID
resp = requests.post(f"{BASE_URL}/api/subscribe", json={
    "email": "test@example.com",
    "feeds": ["nonexistent_feed"]
})
print("Invalid feed ->")
pp(resp)
assert resp.status_code == 400

In [ ]:
# Empty feeds array
resp = requests.post(f"{BASE_URL}/api/subscribe", json={
    "email": "test@example.com",
    "feeds": []
})
print("Empty feeds ->")
pp(resp)
assert resp.status_code == 400

---
## Step 3: Fetch Today's Content

Partners can pull today's newsletter content to display on their own site.
Returns both raw markdown and pre-rendered HTML.

In [4]:
# Fetch Into Crypto CN content
resp = requests.get(f"{BASE_URL}/api/content/today?feed=into_crypto_cn")
pp(resp)

HTTP 404
{
  "error": "No content available"
}


In [ ]:
# Fetch Minor News content
resp = requests.get(f"{BASE_URL}/api/content/today?feed=minor_news")

if resp.status_code == 200:
    data = resp.json()
    print(f"Date: {data['date']}")
    print(f"Title: {data['emoji']} {data['title']}")
    print(f"Content preview: {data['content'][:200]}...")
    print(f"HTML available: {len(data['html'])} chars")
elif resp.status_code == 404:
    print("No content available for today yet.")
    print("Content is generated daily around 07:00 UTC.")
else:
    pp(resp)

In [ ]:
# Invalid feed returns 400 with list of valid feeds
resp = requests.get(f"{BASE_URL}/api/content/today?feed=invalid")
print("Invalid feed ->")
pp(resp)
assert resp.status_code == 400

---
## Step 4: Unsubscribe

Every email includes an unsubscribe link with a unique token.
Partners can also build a custom unsubscribe UI using the API.

In [ ]:
# Missing token
resp = requests.get(f"{BASE_URL}/api/unsubscribe")
print("No token ->")
pp(resp)
assert resp.status_code == 400

In [ ]:
# Invalid token
resp = requests.get(f"{BASE_URL}/api/unsubscribe?token=invalid-token")
print("Invalid token ->")
pp(resp)
assert resp.status_code == 404

In [ ]:
# To test real unsubscribe, paste a token from the subscriptions table:
# UNSUB_TOKEN = "paste-real-token-here"
# resp = requests.get(f"{BASE_URL}/api/unsubscribe?token={UNSUB_TOKEN}")
# pp(resp)

---
## Step 5: Full Integration Example

Here's how a partner would build a complete subscription + content display flow:

In [ ]:
def partner_integration_demo(user_email, selected_feeds):
    """Simulates a partner's backend integration."""
    print(f"=== Partner Integration Demo ===")
    print(f"User: {user_email}")
    print(f"Selected: {selected_feeds}")
    print()

    # 1. Subscribe the user
    resp = requests.post(f"{BASE_URL}/api/subscribe", json={
        "email": user_email,
        "feeds": selected_feeds
    })

    if resp.status_code == 200:
        result = resp.json()
        print(f"[OK] Subscribed to {result['feeds']}")
        print(f"     User will receive the latest issue immediately.")
    else:
        error = resp.json().get("error", "Unknown error")
        print(f"[FAIL] {error}")
        return

    print()

    # 2. Fetch content to display on partner site
    for feed in selected_feeds:
        resp = requests.get(f"{BASE_URL}/api/content/today?feed={feed}")
        if resp.status_code == 200:
            data = resp.json()
            print(f"[OK] {data['emoji']} {data['title']} ({data['date']})")
            print(f"     Content: {len(data['content'])} chars markdown")
            print(f"     HTML:    {len(data['html'])} chars (ready to embed)")
        elif resp.status_code == 404:
            print(f"[--] {feed}: No content today (generated at 07:00 UTC)")
        else:
            print(f"[FAIL] {feed}: HTTP {resp.status_code}")

    print()
    print("Done. User is now subscribed and will receive daily emails at 08:00 UTC.")


# Run the demo
partner_integration_demo(
    user_email="demo@partner-site.com",
    selected_feeds=["minor_news", "into_crypto_cn"]
)

---
## API Reference

| Endpoint | Method | Auth | Description |
|----------|--------|------|-------------|
| `/api/newsletters` | GET | No | List available newsletters |
| `/api/subscribe` | POST | No | Subscribe user (sends latest issue) |
| `/api/content/today?feed=` | GET | No | Get today's content (markdown + HTML) |
| `/api/unsubscribe?token=` | GET | No | Unsubscribe via token |

### Newsletter IDs

| ID | Name | Language |
|----|------|----------|
| `minor_news` | Minor News | English |
| `into_crypto_cn` | Into Crypto 中文版 | Chinese |
| `into_crypto_en` | Into Crypto | English |

### Notes

- **CORS** enabled on all public endpoints — safe to call from browser JS
- **Idempotent** — re-subscribing merges feeds, never duplicates
- **Welcome email** — new subscribers get the latest issue immediately
- **Content schedule** — generated ~07:00 UTC, emails sent ~08:00 UTC